# Always Run

In [20]:
import pandas as pd
import requests
import json
import datetime as dt
from api_keys import key

In [2]:
url = "https://api.the-odds-api.com/v3/odds/?"
sport = "icehockey_nhl"
ml_query_url = f"{url}apiKey={key}&sport={sport}&region=us"
ml_response = requests.get(ml_query_url).json()

In [3]:
def pl_api(data):
    team = []
    date = []
    home_odds = []
    away_odds = []
    odds = []
    
    for game in range(len(data['data'])):
        event = data['data'][game]
        team.append(event['teams'][0])
        team.append(event['teams'][1])
        time = dt.datetime.fromtimestamp(int(event['commence_time'])).strftime('%Y-%m-%d')
        date.append(time)
        date.append(time)
        home_odds = []
        away_odds = []
        for site in range(len(event['sites'])):
            h_odd = event['sites'][site]['odds']['spreads']['odds'][0]
            a_odd = event['sites'][site]['odds']['spreads']['odds'][1]    
            home_odds.append(h_odd)
            away_odds.append(a_odd)
        if event['sites_count'] == 0:
            odds.append('NaN')
            odds.append('NaN')
        else:
            h_odds = sum(home_odds)/len(home_odds)
            a_odds = sum(away_odds)/len(away_odds)
            if h_odds > 2:
                h_odds = round((h_odds - 1) * 100)
            elif h_odds == 1:
                h_odds = 'NaN'
            elif h_odds < 2:
                h_odds = round(100 / (1 - h_odds))
            else:
                h_odds = 100
            if a_odds > 2:
                a_odds = round((a_odds - 1) * 100)
            elif a_odds == 1:
                a_odds = 'NaN'
            elif a_odds < 2:
                a_odds = round(100 / (1 - a_odds))
            else:
                a_odds = 100
            odds.append(h_odds)
            odds.append(a_odds)
        
    game_dict = {
        "Team": team,
        "Date": date,
        "PL": odds
    }
    df = pd.DataFrame(game_dict)
    df = df.round({'PL':0})
    return df

In [21]:
def ml_api(data):
    team = []
    date = []
    home = []
    home_odds = []
    away_odds = []
    odds = []
    
    for game in range(len(data['data'])):
        event = data['data'][game]
        team.append(event['teams'][0])
        team.append(event['teams'][1])
        home.append(1)
        home.append(0)
        time = dt.datetime.fromtimestamp(int(event['commence_time'])).strftime('%Y-%m-%d')
        date.append(time)
        date.append(time)
        home_odds = []
        away_odds = []
        for site in range(len(event['sites'])):
            if event['sites'][site]['site_key']=='unibet':
                if event['sites_count'] == 1:
                    odds.append('NaN')
                    odds.append('NaN')
                else:
                    continue
            else:
                h_odd = event['sites'][site]['odds']['h2h'][0]
                a_odd = event['sites'][site]['odds']['h2h'][1]  
                home_odds.append(h_odd)
                away_odds.append(a_odd)
        if event['sites_count'] == 0:
            odds.append('NaN')
            odds.append('NaN')
        else:
            h_odds = sum(home_odds)/len(home_odds)
            a_odds = sum(away_odds)/len(away_odds)
            if h_odds > 2:
                h_odds = round((h_odds - 1) * 100)
            elif h_odds == 1:
                h_odds = 'NaN'
            elif h_odds < 2:
                h_odds = round(100 / (1 - h_odds))
            else:
                h_odds = 100
            if a_odds > 2:
                a_odds = round((a_odds - 1) * 100)
            elif a_odds == 1:
                a_odds = 'NaN'
            elif a_odds < 2:
                a_odds = round(100 / (1 - a_odds))
            else:
                a_odds = 100
            odds.append(h_odds)
            odds.append(a_odds)
        
    game_dict = {
        "Team": team,
        "Home": home,
        "Date": date,
        "ML": odds
    }
    df = pd.DataFrame(game_dict)
    df = df.round({'ML':0})
    return df

In [22]:
ml_odds = ml_api(ml_response)
ml_odds

,Team,Home,Date,ML
0,Carolina Hurricanes,1,2020-08-01,-126
1,New York Rangers,0,2020-08-01,108
2,Chicago Blackhawks,1,2020-08-01,132
3,Edmonton Oilers,0,2020-08-01,-151
4,Florida Panthers,1,2020-08-01,-100
5,New York Islanders,0,2020-08-01,-118
6,Montreal Canadiens,1,2020-08-01,146
7,Pittsburgh Penguins,0,2020-08-01,-169
8,Calgary Flames,1,2020-08-01,-123
9,Winnipeg Jets,0,2020-08-01,104


In [23]:
stats_url = 'https://www.hockey-reference.com/leagues/NHL_2020.html#all_stats'
tables = pd.read_html(stats_url)
stats_df_1 = tables[0]
stats_df_1 = stats_df_1.drop([0, 9])
stats_df_2 = tables[1]
stats_df_2 = stats_df_2.drop([0, 8])
frames = [stats_df_1, stats_df_2]
stats = pd.concat(frames)
stats = stats.rename(columns={"Unnamed: 0": "Team","W": "season_wins"})
cols = [1,3,4,5,6,9,10,11,12,13,14]
stats.drop(stats.columns[cols],axis=1,inplace=True)
net_g = []
for i in stats.iterrows():
    #net_goals calculation
    net_g.append(int(i[1]['GF']) - int(i[1]['GA']))

stats['net_goals'] = net_g
cols = [2,3]
stats.drop(stats.columns[cols],axis=1,inplace=True)
stats = stats.replace({'Team': {'St. Louis Blues': 'St Louis Blues'}})
stats

,Team,season_wins,net_goals
1,Boston Bruins,44,53
2,Tampa Bay Lightning,43,50
3,Toronto Maple Leafs,36,11
4,Florida Panthers,35,3
5,Montreal Canadiens,31,-9
6,Buffalo Sabres,30,-22
7,Ottawa Senators,25,-52
8,Detroit Red Wings,17,-122
10,Washington Capitals,41,25
11,Philadelphia Flyers,41,36


In [24]:
raw_df = pd.merge(ml_odds,stats,on='Team')
raw_df["PL"] = ""
raw_df["Line Mvmt"] = ""
raw_df = raw_df[["Team","Date","Home", "ML","PL","Line Mvmt","net_goals","season_wins"]]
raw_df

,Team,Date,Home,ML,PL,Line Mvmt,net_goals,season_wins
0,Carolina Hurricanes,2020-08-01,1,-126,,,29,38
1,New York Rangers,2020-08-01,0,108,,,12,37
2,Chicago Blackhawks,2020-08-01,1,132,,,-6,32
3,Edmonton Oilers,2020-08-01,0,-151,,,8,37
4,Florida Panthers,2020-08-01,1,-100,,,3,35
5,New York Islanders,2020-08-01,0,-118,,,-1,35
6,Montreal Canadiens,2020-08-01,1,146,,,-9,31
7,Pittsburgh Penguins,2020-08-01,0,-169,,,28,40
8,Calgary Flames,2020-08-01,1,-123,,,-5,36
9,Winnipeg Jets,2020-08-01,0,104,,,13,37


In [25]:
raw_df.to_csv('first_scrape.csv')

# Run the Second Time Only

In [16]:
df = pd.read_csv('first_scrape.csv')
cols = [0,4]
df.drop(df.columns[cols],axis=1,inplace=True)
df

,Team,Home,ML,PL,Line Mvmt,net_goals,season_wins
0,Carolina Hurricanes,1,-119,196.0,NaN,29,38
1,New York Rangers,0,113,-236.0,NaN,12,37
2,Chicago Blackhawks,1,138,-192.0,NaN,-6,32
3,Edmonton Oilers,0,-143,164.0,NaN,8,37
4,Florida Panthers,1,106,-254.0,NaN,3,35
5,New York Islanders,0,-111,209.0,NaN,-1,35
6,Montreal Canadiens,1,188,NaN,NaN,-9,31
7,Pittsburgh Penguins,0,-128,NaN,NaN,28,40
8,Calgary Flames,1,-117,199.0,NaN,-5,36
9,Winnipeg Jets,0,110,-240.0,NaN,13,37


In [ ]:
pl_query_url = f"{url}apiKey={key}&sport={sport}&region=us&mkt=spreads"
pl_response = requests.get(pl_query_url).json()
pl_odds = pl_api(pl_response)
pl_odds

In [ ]:
df = df.rename(columns={"ML": "Open"})
final_df = pd.merge(pd.merge(df,ml_odds,on=['Team','Date']),pl_odds,on=['Team','Date'])
lm = []
for i in final_df.iterrows():
    #net_goals calculation
    lm.append(int(i[1]['Open']) - int(i[1]['ML']))
final_df["Line Mvmt"] = lm
final_df = final_df[["Team","Date","Home", "ML","PL","Line Mvmt","net_goals","season_wins"]]
final_df

In [ ]:
final_df.to_csv('raw_input.csv')